# Install pyqlib

In [ ]:
# !pip install pyqlib[all]
!pip install --ignore-installed blinker
!pip install pyqlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.7 MB/s eta 0:00:00
   ━━━

# Mount drive with Google colab

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/MASTER-master') # Replace with the actual path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from master import MASTERModel
import pickle
import numpy as np
import time

# Please install qlib first before load the data.
universe = 'csi300' # ['csi300','csi800']
prefix = 'opensource' # ['original','opensource'], which training data are you using
train_data_dir = f'data'

with open(f'/content/drive/MyDrive/MASTER-master/{train_data_dir}/{prefix}/{universe}_dl_train.pkl', 'rb') as f:
    dl_train = pickle.load(f)

with open(f'/content/drive/MyDrive/MASTER-master/{train_data_dir}/{prefix}/{universe}_dl_valid.pkl', 'rb') as f:
    dl_valid = pickle.load(f)

with open(f'/content/drive/MyDrive/MASTER-master/{train_data_dir}/{prefix}/{universe}_dl_test.pkl', 'rb') as f:
    dl_test = pickle.load(f)

print("Data Loaded.")


d_feat = 158
d_model = 256
t_nhead = 4
s_nhead = 2
dropout = 0.5
gate_input_start_index = 158
gate_input_end_index = 221

if universe == 'csi300':
    beta = 5
elif universe == 'csi800':
    beta = 2
# Added by ME
else:
    raise ValueError(f"Unknown universe: {universe}")
# End of addition

n_epoch = 10
lr = 1e-5
GPU = 0
train_stop_loss_thred = 0.95


ic = []
icir = []
ric = []
ricir = []

Data Loaded.


# Training

In [ ]:
######################################################################################
# for seed in [0, 1, 2, 3, 4]:
for seed in [0]:
    model = MASTERModel(
        d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
        beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
        n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
        save_path='/content/drive/MyDrive/MASTER-master/model_2', save_prefix=f'{universe}_{prefix}_{seed}'
    )

    start = time.time()
    # Train
    model.fit(dl_train, dl_valid)

    print("Model Trained.")

    # Test
    predictions, metrics = model.predict(dl_test)

    running_time = time.time()-start

    print('Seed: {:d} \n Time cost : {:.2f} sec'.format(seed, running_time))
    print(metrics)

    # Save the predictions value
    predictions.to_csv("predictions.csv")

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])
######################################################################################

Epoch: 0
Epoch 0, train_loss 1.038404, valid ic 0.0783, icir 0.539, rankic 0.0923, rankicir 0.661.
Epoch: 1
Epoch 1, train_loss 0.996480, valid ic 0.1015, icir 0.750, rankic 0.1122, rankicir 0.823.
Epoch: 2
Epoch 2, train_loss 0.989620, valid ic 0.1050, icir 0.753, rankic 0.1061, rankicir 0.749.
Epoch: 3
Epoch 3, train_loss 0.986657, valid ic 0.0971, icir 0.704, rankic 0.1072, rankicir 0.793.
Epoch: 4
Epoch 4, train_loss 0.984401, valid ic 0.1055, icir 0.792, rankic 0.1131, rankicir 0.830.
Epoch: 5
Epoch 5, train_loss 0.982559, valid ic 0.1001, icir 0.743, rankic 0.1053, rankicir 0.766.
Epoch: 6
Epoch 6, train_loss 0.980627, valid ic 0.0965, icir 0.717, rankic 0.1019, rankicir 0.759.
Epoch: 7
Epoch 7, train_loss 0.979170, valid ic 0.0833, icir 0.593, rankic 0.0947, rankicir 0.704.
Epoch: 8
Epoch 8, train_loss 0.977238, valid ic 0.0953, icir 0.752, rankic 0.0961, rankicir 0.738.
Epoch: 9
Epoch 9, train_loss 0.975841, valid ic 0.0909, icir 0.701, rankic 0.0935, rankicir 0.729.
Model save

# Load and Test

In [ ]:
# ######################################################################################
# for seed in [0]:
#     # param_path = f'model/{universe}_{prefix}_{seed}.pkl'
#     param_path = f'/content/drive/MyDrive/MASTER-master/model/{universe}_{prefix}_{seed}.pkl'

#     print(f'Model Loaded from {param_path}')
#     model = MASTERModel(
#             d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
#             beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
#             n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
#             save_path='', save_prefix=universe
#         )
#     model.load_param(param_path)
#     predictions, metrics = model.predict(dl_test)
#     print(metrics)

#     # Save the predictions value
#     predictions.to_csv("predictions.csv")

#     ic.append(metrics['IC'])
#     icir.append(metrics['ICIR'])
#     ric.append(metrics['RIC'])
#     ricir.append(metrics['RICIR'])

# ######################################################################################

Model Loaded from /content/drive/MyDrive/MASTER-master/model/csi300_opensource_0.pkl
Epoch: 1


/content/drive/My Drive/MASTER-master/base_model.py:158: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(param_path, map_location=self.de

{'IC': 0.06509194126049142, 'ICIR': 0.44007271824478056, 'RIC': 0.06845278603501052, 'RICIR': 0.44897479351814257}


# Performance

In [ ]:
print("IC: {:.4f} pm {:.4f}".format(np.mean(ic), np.std(ic)))
print("ICIR: {:.4f} pm {:.4f}".format(np.mean(icir), np.std(icir)))
print("RIC: {:.4f} pm {:.4f}".format(np.mean(ric), np.std(ric)))
print("RICIR: {:.4f} pm {:.4f}".format(np.mean(ricir), np.std(ricir)))

IC: 0.0595 pm 0.0000
ICIR: 0.3673 pm 0.0000
RIC: 0.0638 pm 0.0000
RICIR: 0.3900 pm 0.0000
